### Тестовое задание

In [1]:
!pip install sqlalchemy

In [2]:
import psycopg2
import pandas as pd

# Устанавливаем соединение
conn = psycopg2.connect(
    dbname='test_db',
    user='test_user',
    password='test_pass',
    host='localhost',
    port=5432
)

# Пишем SQL-запрос
query = '''
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
'''

# Читаем результат запроса в датафрейм
df = pd.read_sql(query, conn)
print(df)


  table_name
0     stores
1    product
2      sales
3  prod_disc


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\444617832.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3]:
pd.read_sql("SELECT * FROM sales LIMIT 5", conn)

C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\3973089995.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM sales LIMIT 5", conn)


,store_id,date_of_day,art_no,sell_qty_colli,sell_val_gsp
0,59,2018-12-22,331716,1.000,253.39
1,356,2018-02-01,482308,4.000,97.82
2,55,2018-04-30,139947,1.000,57.45
3,65,2018-08-08,494125,1.000,465.25
4,18,2017-11-10,128556,1.175,950.66


## Задание №1 — Средняя цена продажи

### Задание №1
###	На основании таблицы с продажами необходимо рассчитать среднюю цену продажи, по следующим критериям:
ТЦ 10, категория Wines
Город Ekaterinburg, категории Sweets и Grocery

Поля в результате:
store_id, cma, art_name, средняя цена продажи



In [4]:
query = '''
SELECT
    s.store_id,
    p.cma,
    p.art_name,
    ROUND(SUM(s.sell_val_gsp) / NULLIF(SUM(s.sell_qty_colli), 0), 2) AS avg_price
FROM sales s
JOIN product p ON s.art_no = p.art_no
JOIN stores st ON s.store_id = st.store_id
WHERE
    (s.store_id = 10 AND p.cma = 'Wines')
    OR
    (st.city_desc = 'Ekaterinburg' AND p.cma IN ('Sweets', 'Grocery'))
GROUP BY s.store_id, p.cma, p.art_name
ORDER BY avg_price DESC;
'''

pd.read_sql(query, conn)


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\315620266.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,store_id,cma,art_name,avg_price


In [5]:
# 📌 Проверка данных по шагам

# 1. Все магазины
query = "SELECT * FROM stores;"
display(pd.read_sql(query, conn))

# 2. Магазины в Екатеринбурге
query = "SELECT * FROM stores WHERE city_desc = 'Ekaterinburg';"
display(pd.read_sql(query, conn))

# 3. Доступные категории товаров
query = "SELECT DISTINCT cma FROM product;"
display(pd.read_sql(query, conn))

# 4. Продажи в магазине с ID = 10
query = "SELECT * FROM sales WHERE store_id = 10;"
display(pd.read_sql(query, conn))

# 5. Продажи товаров категории 'Wines'
query = '''
SELECT s.*, p.cma, p.art_name
FROM sales s
JOIN product p ON s.art_no = p.art_no
WHERE p.cma = 'Wines';
'''
display(pd.read_sql(query, conn))

# 6. Продажи категорий 'Sweets' и 'Grocery' в Екатеринбурге
query = '''
SELECT s.*, p.cma, p.art_name, st.city_desc
FROM sales s
JOIN product p ON s.art_no = p.art_no
JOIN stores st ON s.store_id = st.store_id
WHERE st.city_desc = 'Ekaterinburg'
  AND p.cma IN ('Sweets', 'Grocery');
'''
display(pd.read_sql(query, conn))


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\141075529.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,store_id,city_desc,district
0,10,Moscow,Moscow
1,11,Moscow,Moscow
2,21,Nizhny Novgorod,North-West


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\141075529.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,store_id,city_desc,district


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\141075529.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


...

,store_id,date_of_day,art_no,sell_qty_colli,sell_val_gsp


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\141075529.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,store_id,date_of_day,art_no,sell_qty_colli,sell_val_gsp,cma,art_name


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\141075529.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query, conn))


,store_id,date_of_day,art_no,sell_qty_colli,sell_val_gsp,cma,art_name,city_desc


### Пояснение к заданию №1: Средняя цена продажи

**Условие:**  
На основании таблицы с продажами необходимо рассчитать среднюю цену продажи, по следующим критериям:

- Магазин с `store_id = 10` и категория товара `Wines`
- Город `Ekaterinburg` и категории `Sweets` или `Grocery`

**Ожидаемые поля:**
- `store_id`, `cma`, `art_name`, `avg_price`


###  Почему результат запроса пустой?

Запрос выполнен корректно, однако он не возвращает ни одной строки. Это связано с тем, что **в исходных данных отсутствуют значения**, удовлетворяющие условиям:

- В таблице `product` **нет товаров** с категорией `Wines`  
  → см. результат запроса `SELECT DISTINCT cma FROM product;`

- В таблице `stores` **нет магазинов** в городе `Ekaterinburg`  
  → см. результат запроса `SELECT DISTINCT city_desc FROM stores;`

---

### ✅ Вывод:
Запрос работает корректно, но не находит совпадений, потому что нужных данных (категория `Wines`, город `Ekaterinburg`) просто нет в таблицах.




---

## Задание №2 — Продажи со скидкой и без

### Задание №2
###	На основании таблицы с продажами необходимо рассчитать по категориям сколько было продано единиц товара и на какую сумму, по регулярной цене и в рамках предоставленной скидки.

Поля в результате:
месяц продажи, cma, период со скидкой\без скидки, кол-во единиц товара, сумма продажи


In [ ]:
query = '''
SELECT
    TO_CHAR(s.date_of_day, 'YYYY-MM') AS sale_month,
    p.cma,
    CASE
        WHEN d.art_no IS NOT NULL THEN 'со скидкой'
        ELSE 'без скидки'
    END AS discount_flag,
    ROUND(SUM(s.sell_qty_colli), 2) AS total_qty,
    ROUND(SUM(s.sell_val_gsp), 2) AS total_sales
FROM sales s
JOIN product p ON s.art_no = p.art_no
LEFT JOIN prod_disc d
    ON s.art_no = d.art_no
   AND s.store_id = d.store_id
   AND s.date_of_day BETWEEN d.date_from AND d.date_to
GROUP BY
    TO_CHAR(s.date_of_day, 'YYYY-MM'),
    p.cma,
    discount_flag
ORDER BY
    sale_month,
    p.cma,
    discount_flag;

'''

df_check = pd.read_sql(query, conn)
display(df_check)


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\360905391.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_check = pd.read_sql(query, conn)


,store_id,date_of_day,art_no,date_from,date_to,disc_store_id,disc_art_no


###  Пояснение к заданию №2

Цель — рассчитать продажи по категориям товаров и по месяцам с разбивкой на:

- период **со скидкой** (если дата продажи попадает в диапазон действия скидки)
- период **без скидки**

Агрегирование выполняется:
- по месяцу (`YYYY-MM`)
- по категории (`cma`)
- с расчётом:
  - общего количества проданных единиц (`SUM(sell_qty_colli)`)
  - общей суммы продаж (`SUM(sell_val_gsp)`)

Сравнение выполняется через `LEFT JOIN` `sales` с `prod_disc`, чтобы определить, действовала ли скидка на момент продажи.

⚠️ В тестовом датасете отсутствуют пересечения между `sales` и `prod_disc` по `art_no`, `store_id` и дате — поэтому все продажи классифицированы как «без скидки».




---

## Задание №3 — Скидки на дату

### Задание №3
### 	На основании таблицы с предоставленными скидками необходимо вывести список категорий, артикулов, их наименований, ТЦ, название городов. Рассчитать процент скидки на каждой позиции.
Критерии:
Город Nizhny Novgorod
Категории Drinks, Wines, Bakery
Дата действия скидки 15 марта 2018

Поля в результате:
город, ТЦ, категория, артикул, наименование, % скидки
Оставить только те строки, где скидка превышает 50%



In [7]:
query = '''
SELECT 
    st.city_desc AS city,
    d.store_id,
    p.cma,
    d.art_no,
    p.art_name,
    ROUND((d.old_colli_gsp - d.discount_colli_gsp) / d.old_colli_gsp * 100, 2) AS discount_percent
FROM prod_disc d
JOIN product p ON d.art_no = p.art_no
JOIN stores st ON d.store_id = st.store_id
WHERE DATE '2018-03-15' BETWEEN d.date_from AND d.date_to
  AND st.city_desc = 'Nizhny Novgorod'
  AND p.cma IN ('Drinks', 'Wines', 'Bakery')
  AND (d.old_colli_gsp - d.discount_colli_gsp) / d.old_colli_gsp > 0.5
ORDER BY discount_percent DESC;
'''

pd.read_sql(query, conn)


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\3840533255.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,city,store_id,cma,art_no,art_name,discount_percent


### Пояснение к заданию №3 

Запрос не вернул данных, так как ни одна из записей в таблице `prod_disc` не попадает на дату `15 марта 2018 года`, а также в таблице `product` отсутствуют товары с категориями `Drinks`, `Wines` и `Bakery`. Кроме того, в таблице `stores` только один ТЦ в городе Nizhny Novgorod, и он не участвует в записях со скидками.

Таким образом, все условия задания не выполняются одновременно ни для одной записи.


---

## Задание №4 — Топ-5 артикулов по продажам

### Задание №4
###	На основании таблицы по продажам на уровне региона в каждой категории товара вывести топ 5 артикулов по сумме продаж в рублях. 

Поля в результате:
регион, категория, артикул, сумма продажи



In [8]:
query = '''
WITH ranked_sales AS (
  SELECT 
    st.district AS region,
    p.cma AS category,
    s.art_no,
    ROUND(SUM(s.sell_val_gsp), 2) AS total_sales,
    ROW_NUMBER() OVER (PARTITION BY st.district, p.cma ORDER BY SUM(s.sell_val_gsp) DESC) AS rank
  FROM sales s
  JOIN stores st ON s.store_id = st.store_id
  JOIN product p ON s.art_no = p.art_no
  GROUP BY st.district, p.cma, s.art_no
)
SELECT region, category, art_no, total_sales
FROM ranked_sales
WHERE rank <= 5
ORDER BY region, category, total_sales DESC;
'''

pd.read_sql(query, conn)


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\513382899.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,region,category,art_no,total_sales


### Пояснение к заданию №4 

**Условие:**  
Определить топ-5 артикулов по сумме продаж для каждой категории товара на уровне региона (district).


Для выполнения запроса необходимо соединить таблицы:
- `sales` — продажи по `store_id` и `art_no`
- `product` — категория по `art_no`
- `stores` — регион (`district`) по `store_id`

Однако:

1. В таблице `sales` используются артикула (`art_no`), которых **нет в справочнике `product`** → невозможно определить категорию.
2. Многие `store_id` из `sales` **не представлены в справочнике `stores`** → невозможно определить регион.
3. В результате соединения `sales JOIN product JOIN stores` не формируется ни одной строки.

---

### ✅ Вывод

Запрос написан корректно, однако **в тестовых данных отсутствуют связи между таблицами**, необходимые для выполнения задания:
- отсутствуют артикула-продукты,
- отсутствуют магазины-регионы.

Поэтому результат запроса — пустой.



---

## Задание №5 — Накопительный итог по продажам

### Задание №5
###	На основании таблицы с продажами, посчитать накопитольную сумму продажи за 22 год в разбивке по месяцам.
Поля в результате:
месяц, накопительный итог


In [9]:
query = '''
WITH monthly_sales AS (
    SELECT
        TO_CHAR(date_of_day, 'YYYY-MM') AS month,
        SUM(sell_val_gsp) AS monthly_total
    FROM sales
    WHERE EXTRACT(YEAR FROM date_of_day) = 2022
    GROUP BY TO_CHAR(date_of_day, 'YYYY-MM')
)
SELECT
    month,
    ROUND(SUM(monthly_total) OVER (ORDER BY month), 2) AS cumulative_total
FROM monthly_sales
'''
pd.read_sql(query, conn)


C:\Users\Лилия\AppData\Local\Temp\ipykernel_20400\564347703.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,month,cumulative_total


### Пояснение к Заданию №5
Цель: рассчитать накопительный итог продаж по месяцам за 2022 год.

 Что делает запрос:

Сначала с помощью WITH monthly_sales AS (...) мы собираем суммы продаж (sell_val_gsp) по каждому месяцу — форматируем дату через TO_CHAR(date_of_day, 'YYYY-MM').

Ограничиваем выборку только 2022 годом через EXTRACT(YEAR FROM date_of_day) = 2022.

Считаем накопительную сумму (cumulative_total) с использованием оконной функции SUM(...) OVER (ORDER BY month) — она добавляет каждое последующее значение к предыдущим.

 Почему результат пустой:
В таблице sales, представленной в тестовых данных, отсутствуют продажи за 2022 год — есть только данные за 2017–2018. Поэтому даже корректный запрос возвращает пустую таблицу — это ожидаемо, а не ошибка.